In [1]:
import numpy as np
import pandas as pd
import os
import tqdm
import pickle
from tqdm.notebook import tqdm

from functools import lru_cache

from snorkel.labeling import labeling_function
from snorkel.labeling import LabelingFunction
from snorkel.labeling.model import LabelModel
from snorkel.labeling import PandasLFApplier

#from labeling_functions import get_all_lfs
from pruning_lfs import prune_lfs

import glob

2023-11-07 14:20:15.854191: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-07 14:20:15.855451: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-07 14:20:15.878845: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-07 14:20:15.879547: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-07 14:20:16.287048: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

resources get initialised


Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [2]:
all_lfs = prune_lfs()

/workspace/pruning_lfs.py:52: RuntimeWarning: invalid value encountered in scalar divide
  merged_data.append([index, polarity, cov_simp, cov_src, precision, 1 - precision, cov_simp+cov_src, abs(cov_simp - cov_src), abs(cov_simp - cov_src)/(cov_simp+cov_src)])


In [3]:
len(all_lfs)

328

In [4]:
with open("workspace/datasets/final_combined_with_index.pkl", 'rb') as f:
    dataset = pickle.load(f)

In [5]:
ABSTAIN = -1
SIMPLE = 0
NOT_SIMPLE = 1
LOST_MEANING = 2

label_map = {-1: "ABSTAIN", 0: "SIMPLE", 1: "NOT_SIMPLE", 2: "LOST_MEANING"}

In [6]:
#current_type = "simp"
current_type = "src"

In [7]:
dataset['simplified_snt'] = dataset[current_type]
dataset['source_snt'] = dataset['src']

In [8]:
selected_dataset = "MTurkSF"

In [9]:
dataset = dataset[dataset['ds_id'] == selected_dataset]

In [10]:
dataset

,ds_id,Year,Target_Audience,Domain,index,src,src_id,simp,simp_id,label,...,granularity,duplicated,topic,src_title,simp_title,similarity,topics,val_split,simplified_snt,source_snt
506530,MTurkSF,2022,non-experts,medical,MTurkSF__0__0,Abdominal wall defects are a type of congenita...,0,Abdominal wall defects are a type of congenita...,0,NaN,...,sentence,False,abdominalwalldefect,NaN,NaN,NaN,NaN,False,Abdominal wall defects are a type of congenita...,Abdominal wall defects are a type of congenita...
506531,MTurkSF,2022,non-experts,medical,MTurkSF__0__1,Abdominal wall defects are a type of congenita...,0,Abdominal wall defects are a type of congenita...,1,NaN,...,sentence,False,abdominalwalldefect,NaN,NaN,NaN,NaN,False,Abdominal wall defects are a type of congenita...,Abdominal wall defects are a type of congenita...
506532,MTurkSF,2022,non-experts,medical,MTurkSF__0__2,Abdominal wall defects are a type of congenita...,0,Abdominal wall defects are a type of congenita...,2,NaN,...,sentence,False,abdominalwalldefect,NaN,NaN,NaN,NaN,False,Abdominal wall defects are a type of congenita...,Abdominal wall defects are a type of congenita...
506533,MTurkSF,2022,non-experts,medical,MTurkSF__0__3,Abdominal wall defects are a type of congenita...,0,Abdominal wall defects are a type of congenita...,3,NaN,...,sentence,False,abdominalwalldefect,NaN,NaN,NaN,NaN,False,Abdominal wall defects are a type of congenita...,Abdominal wall defects are a type of congenita...
506534,MTurkSF,2022,non-experts,medical,MTurkSF__0__4,Abdominal wall defects are a type of congenita...,0,Abdominal wall defects are a type of congenita...,4,NaN,...,sentence,False,abdominalwalldefect,NaN,NaN,NaN,NaN,False,Abdominal wall defects are a type of congenita...,Abdominal wall defects are a type of congenita...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506746,MTurkSF,2022,non-experts,medical,MTurkSF__62__216,Visual impairment may cause difficulties with ...,62,Visual impairment may cause difficult with nor...,216,NaN,...,sentence,False,visual-impairment,NaN,NaN,NaN,NaN,False,Visual impairment may cause difficulties with ...,Visual impairment may cause difficulties with ...
506747,MTurkSF,2022,non-experts,medical,MTurkSF__62__217,Visual impairment may cause difficulties with ...,62,Visual impairment may cause trouble with norma...,217,NaN,...,sentence,False,visual-impairment,NaN,NaN,NaN,NaN,False,Visual impairment may cause difficulties with ...,Visual impairment may cause difficulties with ...
506748,MTurkSF,2022,non-experts,medical,MTurkSF__60__218,"Visual impairment, also known as vision impair...",60,"Visual impairment, also known as vision impair...",218,NaN,...,sentence,False,visual-impairment,NaN,NaN,NaN,NaN,False,"Visual impairment, also known as vision impair...","Visual impairment, also known as vision impair..."
506749,MTurkSF,2022,non-experts,medical,MTurkSF__60__219,"Visual impairment, also known as vision impair...",60,"Visual impairment, also known as vision impair...",219,NaN,...,sentence,False,visual-impairment,NaN,NaN,NaN,NaN,False,"Visual impairment, also known as vision impair...","Visual impairment, also known as vision impair..."


In [11]:
label_path = f"/workspace/datasets/{selected_dataset}labels/"

In [12]:
if not os.path.isdir(label_path):
    os.mkdir(label_path)

In [19]:
def get_finished_batches(path = f"/workspace/datasets/{selected_dataset}labels/*"):
    label_paths = glob.glob(path)
    fin_batches = [int(path.split("_")[-1]) for path in label_paths if "labels_" in path and current_type in path]
    return set(fin_batches)

In [20]:
def save_used_lfs(all_lfs, path = f"/workspace/datasets/{selected_dataset}labels/used_lfs.pkl"):
    names = [lf.name for lf in all_lfs]
    pickle.dump(names, open(path, "wb"))

In [21]:
batch_size = 20
start = 0

save_used_lfs(all_lfs)

for i in tqdm(range(start, len(dataset), batch_size), position=1):
    if not i in get_finished_batches():
        try:
            applier = PandasLFApplier(all_lfs)
            labels = applier.apply(dataset[i:i+batch_size], progress_bar=True)
            
            pickle.dump(labels, open(f"/workspace/datasets/{selected_dataset}labels/labels_{current_type}_{i}", "wb"))
            print(f"finished on {i}/{len(dataset)}")

        except Exception as e:
            print(f"something went wrong with batch {i}")
            print(e)

  0%|          | 0/12 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:11<00:00,  1.77it/s]


finished on 0/221


100%|██████████| 20/20 [00:11<00:00,  1.80it/s]


finished on 20/221


...

100%|██████████| 20/20 [00:11<00:00,  1.76it/s]


finished on 60/221


100%|██████████| 20/20 [00:11<00:00,  1.78it/s]


finished on 80/221


 75%|███████▌  | 15/20 [00:08<00:03,  1.63it/s]

In [16]:
#build ds labels
ds_label_path = f"/workspace/datasets/ds_labels"
if not os.path.isdir(ds_label_path):
    os.mkdir(ds_label_path)

In [17]:
def build_labels(ds="MTurkSF"):
    simp_paths = sorted(glob.glob(f"workspace/datasets/{ds}labels/labels_simp*"), key=lambda x: int(x.split("_")[-1]))
    src_paths = sorted(glob.glob(f"workspace/datasets/{ds}labels/labels_src*"), key=lambda x: int(x.split("_")[-1]))

    simp_labels = [pickle.load(open(path, "rb")) for path in simp_paths]
    src_labels = [pickle.load(open(path, "rb")) for path in src_paths]

    simp_labels = np.concatenate(simp_labels)
    src_labels = np.concatenate(src_labels)

    pickle.dump(simp_labels, open(f"{ds_label_path}/{ds}_simp_labels.pkl", "wb"))
    pickle.dump(src_labels, open(f"{ds_label_path}/{ds}_src_labels.pkl", "wb"))

In [ ]:
selected_dataset


In [ ]:
build_labels(selected_dataset)